In [52]:
from wellplate.elements import read_plate_xml

well_map, info, labels = read_plate_xml('C:\Johan-test.xml')
display(well_map)

,well,EP,AB,Note
0,A1,A,AB2,Testing
1,A2,A,AB2,Testing
2,A3,A,AB2,Testing
3,A4,A,AB2,Testing
4,A5,A,AB2,Testing
...,...,...,...,...
91,H8,B,AB1,None
92,H9,B,AB1,None
93,H10,B,AB1,None
94,H11,B,AB1,None


In [48]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

def read_plate_xml(file_loc):
    # get tree
    tree = ET.parse(file_loc)
    root = tree.getroot()
    # get active preset.
    preset_index = int(root.find("Presets/PresetIndex").text)
    # read all label info.
    labels = read_labels(root,preset_index)
    # read all meta data.
    meta_data,meta_data_info = read_meta_data(root,preset_index)
    return meta_data, meta_data_info, labels

def read_labels(root, preset_index):
    labels = []
    for elem in root.findall(f"./WellplateMetadata_{preset_index}/Labels/"):
        # get tag info.
        label_name = elem.find('Name').text
        label_color = elem.find('Color').text
        # get well info.
        search_str = f"./WellplateMetadata_1/Labels/Label[Name='{label_name}']/Wells/WellIndex"
        label_wells_ind = np.unique(np.array([int(elem.text) for elem in root.findall(search_str)]))
        label_wells_id = [well_ind_to_id(ind) for ind in label_wells_ind]
        labels.append({'name':label_name, 'tag_color':label_color,'well_indices':label_wells_ind,'well_ids':label_wells_id})
    labels = pd.DataFrame(labels).drop_duplicates(subset='name')
    return labels

def read_meta_data(root, preset_index):
    meta_data = pd.DataFrame([])
    # get well IDs.
    meta_data['well'] = [ well_ind_to_id(i) for i in range(96)]
    meta_data_info = []
    for elem in root.findall(f"./WellplateMetadata_{preset_index}/Quantities/Quantity"):
        # get meta info.
        meta_name = elem.find('Name').text
        meta_desc = elem.find('Desc').text
        meta_data_info.append({'name':meta_name,'desc':meta_desc})
        # get value each well.
        meta_wells = np.empty(96,object)
        for well in elem.findall('Wells/Well'):
            well_index = int(well.find('WellIndex').text)
            meta_wells[well_index] = well.find('Quality').text
        meta_data[meta_name] = meta_wells
    return meta_data, meta_data_info

def well_ind_to_id(ind):
    row_col = np.unravel_index(ind,(8,12))
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    well_id = f"{letters[row_col[0]]}{row_col[1]+1}"
    return well_id

meta_data,meta_data_info, labels = read_plate_xml('C:\Johan-test.xml')
display(labels)
display(meta_data)
display(meta_data_info)
print()

,name,tag_color,well_indices,well_ids
0,ct1-HA,FFFF88FF,"[6, 7, 8, 9, 10, 11]","[A7, A8, A9, A10, A11, A12]"
1,ct3-HA,FF006666,"[18, 19, 20, 21, 22, 23]","[B7, B8, B9, B10, B11, B12]"
2,ct1-ALFA,FFF3A345,"[0, 1, 2, 3, 4, 5]","[A1, A2, A3, A4, A5, A6]"
3,ct1-GLU2,FF00FF00,"[24, 25, 26, 27, 28, 29]","[C1, C2, C3, C4, C5, C6]"
4,ct1-OLLAS,FF8888FF,"[30, 31, 32, 33, 34, 35, 57, 58, 59]","[C7, C8, C9, C10, C11, C12, E10, E11, E12]"
5,ct1-PRTC,FFF3A345,"[48, 49, 50, 51, 52, 53, 54, 55, 56]","[E1, E2, E3, E4, E5, E6, E7, E8, E9]"
6,ct1-VSV,FFFF00FF,"[72, 73, 74, 75, 76, 77, 78, 79, 80]","[G1, G2, G3, G4, G5, G6, G7, G8, G9]"
7,ct3-ALFA,FF636345,"[12, 13, 14, 15, 16, 17]","[B1, B2, B3, B4, B5, B6]"
8,ct3-GLU2,FF006600,"[36, 37, 38, 39, 40, 41]","[D1, D2, D3, D4, D5, D6]"
9,ct3-OLLAS,FF8888FF,"[42, 43, 44, 45, 46, 47, 69, 70, 71]","[D7, D8, D9, D10, D11, D12, F10, F11, F12]"


,well,EP,AB,Note
0,A1,A,AB2,Testing
1,A2,A,AB2,Testing
2,A3,A,AB2,Testing
3,A4,A,AB2,Testing
4,A5,A,AB2,Testing
...,...,...,...,...
91,H8,B,AB1,None
92,H9,B,AB1,None
93,H10,B,AB1,None
94,H11,B,AB1,None


[{'name': 'EP', 'desc': 'Epitope'},
 {'name': 'AB', 'desc': 'Antibody'},
 {'name': 'Note', 'desc': 'Notes'}]

In [34]:
preset_index=4


display(meta_data)

EP: Epitope
['A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B']
AB: Antibody
['AB2' 'AB2' 'AB2' 'AB2' 'AB2' 'AB2' 'AB2' 'AB2' 'AB2' 'AB2' 'AB2' 'AB2'
 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1'
 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1'
 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1'
 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1'
 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1'
 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1'
 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1' 'AB1']

,name,desc,well_values
0,EP,Epitope,"[A, A, A, A, A, A, A, A, A, A, A, A, B, B, B, ..."
1,AB,Antibody,"[AB2, AB2, AB2, AB2, AB2, AB2, AB2, AB2, AB2, ..."
2,Note,Notes,"[Testing, Testing, Testing, Testing, Testing, ..."
